In [13]:
import re
import numpy as np
import pandas as pd
from scml.nlp import to_ascii_str
import mylib

In [14]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 400)

In [15]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [16]:
p = re.compile(r'^b"(.*)"$')
def remove_byte_string_syntax(s: str) -> str:
    m = p.match(s)
    if m is None:
        return s
    return m.group(1)


def preprocess(row) -> str:
    res = row["title"]
    res = remove_byte_string_syntax(res)
    res = to_ascii_str(res)
    return res

In [17]:
%%time
train["title_p"] = train.apply(preprocess, axis=1)

CPU times: user 234 ms, sys: 62.5 ms, total: 297 ms
Wall time: 278 ms


In [18]:
cols = ["title", "title_p", "posting_id", "image"]
train[cols].head()

,title,title_p,posting_id,image
0,Paper Bag Victoria Secret,Paper Bag Victoria Secret,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg
1,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE","Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg
2,Maling TTS Canned Pork Luncheon Meat 397 gr,Maling TTS Canned Pork Luncheon Meat 397 gr,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg
3,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg
4,Nescafe \xc3\x89clair Latte 220ml,Nescafe \xc3\x89clair Latte 220ml,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg


In [19]:
train[cols].to_csv("output/tprep.tsv", sep="\t", index=False)